# The Faust box API in DawDreamer

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/DBraun/DawDreamer/blob/main/examples/box_api/Faust%20Box%20API.ipynb)

DawDreamer provides Python bindings to the Faust box API, which is an intermediate representation of Faust. This creates many opportunities and even has some advantages over writing Faust code.
* Anything you can do with Faust you can do with the box API thanks to the `boxFromDSP` function.
* Python scripting can lead to more optimized DSP code. For example, in Faust if you have a reverb effect with a "bypass" parameter, the reverb effect is **always** computing. You are just mixing in 0% of it. With the box API, you can programatically connect/disconnect the reverb effect, leading to faster DSP code!
* You can evaluate boxes in DawDreamer, pass audio to them, and automate their parameters over time. It even works for polyphonic instruments.
* You can export boxes to C++ code (`boxToSource(box, "cpp", "MyDSP")`). More target languages, which we call backends, will be supported over time.
* The relative simplicity of the box API is suitable for evolutionary algorithms and programmatic exploration of DSP blocks.

What follows is a Python-specific adaptation of [Faust's tutorial of the C++ box API](https://faustdoc.grame.fr/tutorials/box-api/).

## Using the box API

The box API grants access to an intermediate stage of Faust compilation chain. In this tutorial, we present the API with Python bindings. The goal is to show how new audio DSP languages (textual or graphical) and algorithms could be built on top of the box API, and benefit from the Faust compiler infrastructure.

## Faust compiler structure

The Faust compiler is composed of several steps:
![compilation-chain.png](https://faustdoc.grame.fr/tutorials/box-api/img/compilation-chain.png)
<center><i>The compilation chain</i></center>

Starting from the DSP source code, the *Semantic Phase* produces signals as conceptually infinite streams of samples or control values. Those signals are then compiled in imperative code (C/C++, LLVM IR, WebAssembly, etc.) in the *Code Generation Phase*.

The *Semantic Phase* itself is composed of several steps:

![semantic-phase.png](https://faustdoc.grame.fr/tutorials/box-api/img/semantic-phase.png)
<center><i>The semantic phase</i></center>

The initial DSP code using the Block Diagram Algebra (BDA) is translated in a flat circuit in normal form in the *Evaluation, lambda-calculus* step.

The list of output signals is produced by the *Symbolic Propagation* step. Each output signal is then simplified and a set of optimizations are done (normal form computation and simplification, delay line sharing, typing, etc.) to finally produce a *list of output signals in normal form*.

The *Code Generation Phase* translates the signals in an intermediate representation named FIR (Faust Imperative Representation) which is then converted to the final target language (C/C++, LLVM IR, WebAssembly,etc.) with a set of backends.

## Accessing the box stage

A new intermediate public entry point has been created in the *Semantic Phase*, after the *Evaluation, lambda-calculus step* to allow the creation of a box expression. The box API (or the C box API version) allows to programmatically create the box expression, then compile it to create a ready-to-use DSP as a C++ class, or LLVM, Interpreter or WebAssembly factories, to be used with all existing architecture files. Several optimizations done at the signal stage will be demonstrated looking at the generated C++ code.

Note that the [signal API](https://dirt.design/DawDreamer/_generate/dawdreamer.faust.signal.html) allows to access another stage in the compilation process.

## Compiling box expressions

To use the box API, the following steps must be taken:

* creating a global compilation context using the `createLibContext` function

* creating a box expression using the box API, progressively building more complex expressions by combining simpler ones

* compiling the box expression with a DawDreamer FaustProcessor `faust_processor.compile_box` (or compile to source code in a target language `boxToSource`)

* finally destroying the compilation context using the `destroyLibContext` function

## Tools

Let's first import libraries and define helper functions.

In [ ]:
# Uncomment the line below while keeping the "!" if you need to install libraries (like on Colab)
# !pip install dawdreamer numpy librosa scipy ipython

import dawdreamer as daw
from dawdreamer.faust import createLibContext, destroyLibContext
from dawdreamer.faust.box import *

import numpy as np
import librosa
from scipy.io import wavfile
from IPython.display import Audio
import IPython.display as ipd

SAMPLE_RATE = 44100

def show_audio(data, autoplay=False):
    if abs(data).max() >= 1.:
        print('normalizing to max: ', abs(data).max())
        data /= abs(data).max()
    
    ipd.display(Audio(data=data, rate=SAMPLE_RATE, normalize=False, autoplay=autoplay))
    

engine = daw.RenderEngine(SAMPLE_RATE, 1)

playback_processor = engine.make_playback_processor("playback", np.zeros((2,1)))
faust_processor = engine.make_faust_processor("faust")

faust_processor.set_dsp_string("process = si.bus(2);")

engine.load_graph([
    (playback_processor, []),
    (faust_processor, ["playback"])
])

def render(duration=5, display=True):
    engine.render(duration)
    audio = engine.get_audio()
    if display:
        show_audio(audio)
    return audio

## Examples

For each example, the equivalent Faust DSP program and SVG diagram are given as helpers. The SVG diagram shows the result of the compilation *propagate* step (so before any of the signal normalization steps).

### Expression generating constant signals

Let's create a program generating a parallel construction of 7 and 3.14 constant values. Here is the Faust DSP code:

![exfaust0.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust0/exfaust0.svg)

```faust
process = 7,3.14;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust0/exfaust0.dsp)

The following code creates a box expression, containing a box `boxPar(boxInt(7), boxReal(3.14))` expression, then compiles it and displays the C++ class:

In [ ]:
createLibContext()

box = boxPar(boxInt(7), boxReal(3.14))
             
faust_processor.compile_box(box)

print(render(display=False))

The C+ code is then:

In [ ]:
print(boxToSource(box, 'cpp', 'MyDSP'))

destroyLibContext()

## Doing some mathematical operations on an input signal

Here is a simple program doing a mathematical operation on an signal input:

![https://faustdoc.grame.fr/tutorials/box-api/exfaust1/exfaust1.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust1/exfaust1.svg)

```faust
process = _,3.14 : +;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust1/exfaust1.dsp)

In [ ]:
createLibContext()

box = boxAdd(boxWire(), boxReal(3.14))
             
faust_processor.compile_box(box)
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

playback_processor.set_data(np.zeros((faust_processor.get_num_input_channels(),1)))

print(render(display=False))

destroyLibContext()

## Defining a delay expression

Here is a simple program delaying the first input:

![exfaust2.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust2/exfaust2.svg)

```faust
process = @(_,7);
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust2/exfaust2.dsp)

The prefix-notation `boxDelay(x, y)` operator is used to delay the `boxWire()` first parameter with the second `boxInt(7)`:

In [ ]:
createLibContext()

box = boxDelay(boxWire(), boxInt(7))
             
faust_processor.compile_box(box)
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

# Our data is only one sample long, so all samples out of bounds are read as zero.
data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[0,:10])

destroyLibContext()

Several options of the Faust compiler allow control of the generated C++ code. By default computation is done sample by sample in a single loop. But the compiler can also generate vector and parallel code. The following code show how to compile in vector mode:

In [ ]:
createLibContext()

box = boxDelay(boxWire(), boxInt(7))
             
faust_processor.compile_box(box, ["-vec", "-lv", "1"])
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

# Our data is only one sample long, so all samples out of bounds are read as zero.
data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[0,:10])

print(boxToSource(box, 'cpp', 'MyDSP'))

destroyLibContext()

And can possibly be faster if the C++ compiler can auto-vectorize it.

If the delay operators are used on the input signal *before* the mathematical operations, then a *single delay* line will be created, taking the maximum size of both delay lines:

![exfaust3.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust3/exfaust3.svg)

```faust
process = _ <: @(500) + 0.5, @(3000) * 1.5;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust3/exfaust3.dsp)

And built with the following code:

In [ ]:
createLibContext()

box = boxSplit(boxWire(), 
               boxPar(boxAdd(boxDelay(boxWire(), 
                                      boxReal(500)), boxReal(0.5)),
                      boxMul(boxDelay(boxWire(), 
                                      boxReal(3000)), boxReal(1.5))))

In the `void compute` method, the single fVec1 delay line is read at 2 differents indices:

In [ ]:
faust_processor.compile_box(box)

print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

print(boxToSource(box, 'cpp', 'MyDSP'))

destroyLibContext()

## Equivalent box expressions

It is really important to note that *syntactically equivalent box expressions* will be *internally represented by the same memory structure* (using hash consing), thus treated in the same way in the further compilations steps. So the following code where the `s1` variable is created to define the `boxAdd(boxDelay(boxWire(), boxReal(500)), boxReal(0.5))` expression, then used in both outputs:

In [ ]:
createLibContext()

b1 = boxAdd(boxDelay(boxWire(), boxReal(500)), boxReal(0.5))
box = boxPar(b1, b1)

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())

destroyLibContext()

Will behave exactly the same as the following code, where the `boxAdd(boxDelay(boxWire(), boxReal(500)), boxReal(0.5))` expression is used twice:

In [ ]:
createLibContext()

box = boxPar(boxAdd(boxDelay(boxWire(), boxReal(500)), boxReal(0.5)),
             boxAdd(boxDelay(boxWire(), boxReal(500)), boxReal(0.5)))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())

destroyLibContext()

## Using User Interface items

User Interface items can be used, as in the following example, with an `hslider`:

![exfaust4.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust4/exfaust4.svg)

```faust
process = _,hslider("Freq [midi:ctrl 7][style:knob]", 100, 100, 2000, 1) : *;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust4/exfaust4.dsp)

Built with the following code:

In [ ]:
createLibContext()

box = boxMul(boxWire(), 
             boxHSlider("Freq [midi:ctrl 7][style:knob]", 
                        boxReal(100), 
                        boxReal(100), 
                        boxReal(2000), 
                        boxReal(1)))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())

destroyLibContext()

With the box API, the layout can be defined using the [labels-as-pathnames](https://faustdoc.grame.fr/manual/syntax/#labels-as-pathnames) syntax, as in the following example:

![exfaust5.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust5/exfaust5.svg)

```faust
import("stdfaust.lib"); 
freq = vslider("h:Oscillator/freq", 440, 50, 1000, 0.1); 
gain = vslider("h:Oscillator/gain", 0, 0, 1, 0.01); 
process = freq*gain;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust5/exfaust5.dsp)

In [ ]:
createLibContext()

box = boxMul(boxVSlider("h:Oscillator/freq",
                        440.,
                        50.,
                        1000.,
                        0.1),
             boxVSlider("h:Oscillator/gain",
                        0.,
                        0.,
                        1.,
                        0.01))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

destroyLibContext()

## Defining recursive signals

Recursive signals can be defined using the boxRec expression. A one sample delay is automatically created to produce a valid computation. Here is a simple example:

![exfaust6.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust6/exfaust6.svg)

```faust
process = + ~ _;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust6/exfaust6.dsp)

Let's analyze why this works. We can verbosely refactor `process = + ~ _;` into:
```faust
A = +; // 2 inputs / 1 output
B = _; // 1 input  / 1 output
process = A~B;
```
The recursive composition `A~B` works because it satisfies the two necessary properties:
* $outputs(A) ≥ inputs(B)$
* $inputs(A) ≥ outputs(B)$

The resulting box output will have $outputs(A)$ outputs and ($inputs(A)-inputs(B)$) inputs

## Accessing the global context

In Faust, the underlying audio engine sample rate and buffer size is accessed using the foreign function and constant mechanism. The values can also be used in the box language with the following helper functions:
* `boxBufferSize()`
* `boxSampleRate()`

So the following DSP program:
```faust
import("stdfaust.lib"); 
process = ma.SR, ma.BS;
```

results in two output channels, the first being the constant sample rate, and the second being the buffer size we chose when creating the Faust processor.

In [ ]:
createLibContext()

box = boxPar(boxSampleRate(), boxBufferSize())

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[:,:3].tolist())

destroyLibContext()

## Creating tables

Read only and read/write tables can be created. The *read-only table* signal is created with `boxReadOnlyTable` and takes:

* a size first argument
* a content second argument
* a read index third argument (between 0 and size-1)

and produces the indexed table content as its single output. The following simple DSP example:

![exfaust8.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust8/exfaust8.svg)

```faust
process = 10,1,int(_) : rdtable;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust8/exfaust8.dsp)

Can be written with the code:

In [ ]:
createLibContext()

box = boxReadOnlyTable(boxInt(10), boxInt(1), boxIntCast(boxWire()))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[:,:3].tolist())

destroyLibContext()

The *read/write table* signal is created with `boxWriteReadTable` and takes:
* a size first argument
* a content second argument
* a write index a third argument (between 0 and size-1)
* the input of the table as fourth argument
* a read index as fifth argument (between 0 and size-1)

and produces the indexed table content as its single output. The following DSP example:

![exfaust9.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust9/exfaust9.svg)

```faust
process = 10,1,int(_) : rdtable;
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust9/exfaust9.dsp)

Can be written with the code:

In [ ]:
createLibContext()

box = boxWriteReadTable(boxInt(10),
                        boxInt(1), 
                        boxIntCast(boxWire()), 
                        boxIntCast(boxWire()), 
                        boxIntCast(boxWire()))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[:,:5].tolist())

destroyLibContext()

## Creating waveforms

The following DSP program defining a waveform:

![exfaust10.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust10/exfaust10.svg)

```faust
process = waveform { 0, 100, 200, 300, 400 };
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust10/exfaust10.dsp)

Can be written with the code, where the size of the waveform is the first output, and the waveform content itself is the second output created with `boxWaveform`, to follow the [waveform semantic](https://faustdoc.grame.fr/manual/syntax/#waveform-primitive):

In [ ]:
createLibContext()

box = boxWaveform([0, 100, 200, 300, 400])

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

print(render(display=False)[:,:8].tolist())

destroyLibContext()

## Creating soundfiles

The [soundfile](https://faustdoc.grame.fr/manual/syntax/#soundfile-primitive) primitive allows access to a list of externally defined sound resources, described as the list of their filename, or complete paths. It takes:

* the sound number (as a integer between 0 and 255 as a [constant numerical expression](https://faustdoc.grame.fr/manual/syntax/#constant-numerical-expressions))
* the read index in the sound (which will access the last sample of the sound if the read index is greater than the sound length)

The generated block has:

* two fixed outputs: the first one is the currently accessed sound length in frames, the second one is the currently accessed sound nominal sample rate
* several more outputs for the sound channels themselves, as a [constant numerical expression](https://faustdoc.grame.fr/manual/syntax/#constant-numerical-expressions)

The soundfile block is created with `boxSoundfile`. Thus the following DSP code:

![exfaust11.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust11/exfaust11.svg)

```faust
process = 0,0 : soundfile("sound[url:{'tango.wav'}]", 1);
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust11/exfaust11.dsp)

Will be created using the box API with:

In [ ]:
createLibContext()

box = boxSoundfile("sound[url:{'tango.wav'}]", 
                   boxInt(2),
                   boxInt(0),
                   boxInt(0))

print(getBoxType(box))

destroyLibContext()

## Using the boxFromDSP function

Complete DSP programs can be compiled to boxes using the `boxFromDSP` function, which takes a DSP program as a string while returning the created box and the number of inputs/outputs:

In [ ]:
createLibContext()

box, inputs, outputs = boxFromDSP("process = os.osc(440) * .1;")

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = np.ones((faust_processor.get_num_input_channels(),1))
playback_processor.set_data(data)

render(display=True)

destroyLibContext()

The resulting box expression can possibly be reused in a more complex construction, as in the following example, where a filter is created using the `boxFromDSP` function, then called with a slider to control its frequency, and the actual input:

In [ ]:
createLibContext()

# Create the filter without parameter.
# boxfilter has two inputs (cutoff frequency and signal input) and one output
boxfilter = boxFromDSP("process = fi.lowpass(5);")[0]

# Create the filter parameters and connect
cutoff = boxHSlider("cutoff", boxReal(300), boxReal(100), boxReal(2000), boxReal(0.01))
cutoffAndInput = boxPar(cutoff, boxWire())

# cutoffAndInput has two outputs, so we can sequentially connect it to boxfilter which needs 2 two inputs.
filteredInput = boxSeq(cutoffAndInput, boxfilter)

# the final box has one input and one output
box = filteredInput

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = -1.+2.*np.random.random((faust_processor.get_num_input_channels(),int(SAMPLE_RATE*5.)))

print('audio input:')
show_audio(data)
playback_processor.set_data(data)
print('audio output:')
render(display=True)

destroyLibContext()

## Defining more complex expressions: phasor and oscillator

More complex signal expressions can be defined, creating boxes using auxiliary definitions. So the following DSP program:

![exfaust12.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust12/exfaust12.svg)

```faust
import("stdfaust.lib");
process = phasor(440)
with {
     decimalpart(x) = x-int(x);
     phasor(f) = f/ma.SR : (+ : decimalpart) ~ _;
};
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust12/exfaust12.dsp)

Can be built using the following helper functions:

In [ ]:
def decimalPart() -> Box:
    return boxSub(boxWire(), boxIntCast(boxWire()))

def phasor(freq: Box) -> Box:
    return boxSeq(boxDiv(freq, boxSampleRate()), 
                  boxRec(boxSplit(boxAdd(), decimalPart()), boxWire()))

And the main function combining them:

In [ ]:
createLibContext()

box = phasor(boxReal(440))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = -1.+2.*np.random.random((faust_processor.get_num_input_channels(),int(SAMPLE_RATE*5.)))

playback_processor.set_data(data)
print('audio output:')
render(display=True)

destroyLibContext()

Now the following oscillator:
    
![exfaust13.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust13/exfaust13.svg)

```faust
import("stdfaust.lib");
process = osc(440), osc(440)
with {
    decimalPart(x) = x-int(x);
    phasor(f) = f/ma.SR : (+ : decimalPart) ~ _;
    osc(f) = sin(2 * ma.PI * phasor(f));
};
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust13/exfaust13.dsp)

Can be built with:

In [ ]:
def osc(freq: Box) -> Box:
    return boxSin(boxMul(boxMul(boxReal(2.0), boxReal(3.141592653)), phasor(freq)))

createLibContext()

box = boxPar(osc(boxReal(440)), osc(boxReal(440)))

print(getBoxType(box))
faust_processor.compile_box(box)
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = -1.+2.*np.random.random((faust_processor.get_num_input_channels(),int(SAMPLE_RATE*5.)))

playback_processor.set_data(data)
print('audio output:')
render(display=True)

print(boxToSource(box, 'cpp', 'MyDSP'))

destroyLibContext()

Which produces a compute method, where one can see that since the same oscillator signal is used on both outputs, it is actually computed once and copied twice.

## Polyphonic MIDI controllable simple synthesizer

Here is a MIDI controlable simple synthesizer, first with the DSP code:

![exfaust15.svg](https://faustdoc.grame.fr/tutorials/box-api/exfaust15/exfaust15.svg)

```faust
import("stdfaust.lib");
process = organ, organ
with {
    decimalPart(x) = x-int(x);
    phasor(f) = f/ma.SR : (+ : decimalPart) ~ _;
    osc(f) = sin(2 * ma.PI * phasor(f));
    freq = nentry("freq", 100, 100, 3000, 0.01);
    gate = button("gate");
    gain = nentry("gain", 0.5, 0, 1, 0.01);
    organ = gate * (osc(freq) * gain + osc(2 * freq) * gain);

};
```

[Try it Yourself >>](https://faustide.grame.fr/?code=https://faustdoc.grame.fr/tutorials/box-api/exfaust15/exfaust15.dsp)

In [ ]:
def osc(freq: Box) -> Box:
    return boxSin(boxMul(boxMul(boxReal(2.0), boxReal(np.pi)), phasor(freq)))

createLibContext()

# Follow the freq/gate/gain convention, 
# see: https://faustdoc.grame.fr/manual/midi/#standard-polyphony-parameters
freq = boxHSlider("freq", 
                   boxReal(100), 
                   boxReal(100), 
                   boxReal(3000), 
                   boxReal(0.01))

gate = boxButton("gate")

gain = boxHSlider("gain",
                   boxReal(.1), 
                   boxReal(0.), 
                   boxReal(1), 
                   boxReal(0.01))

organ = boxMul(gate, boxAdd(boxMul(osc(freq), gain), 
                            boxMul(osc(boxMul(freq, boxInt(2))), gain)))
# Stereo
box = boxPar(organ, organ)

print(getBoxType(box))
faust_processor.num_voices = 10
faust_processor.compile_box(box)
destroyLibContext()

# Note that because we're using polyphony, the frequency/gain/gate parameters are hidden from
# get_parameters_description(). They're not automatable because the MIDI information fully controls them.
print(faust_processor.get_parameters_description())
print(f"Num inputs: {faust_processor.get_num_input_channels()}, outputs: {faust_processor.get_num_output_channels()}")

data = -1.+2.*np.random.random((faust_processor.get_num_input_channels(),int(SAMPLE_RATE*5.)))

playback_processor.set_data(data)

faust_processor.clear_midi()
faust_processor.add_midi_note(60, 127, 0.0, 0.2)
faust_processor.add_midi_note(62, 127, 0.2, 0.2)
faust_processor.add_midi_note(64, 127, 0.4, 0.2)
faust_processor.add_midi_note(65, 127, 0.6, 0.8)
faust_processor.add_midi_note(69, 127, 0.6, 0.8)
faust_processor.add_midi_note(72, 127, 0.6, 0.8)

print('audio output:')
render(display=True)
